# Pipeline de Ensemble por Modelos (SA → MA)

Este notebook detalha, passo a passo, o processo de ensemble em dois níveis:

- Single-Architecture (SA): ensemble por modelo, nos níveis Tile → Image → Patient.
- Multi-Architecture (MA): ensemble entre diferentes modelos, nos níveis Image → Tile → Patient.

Ao final, os artefatos (CSVs e métricas) são organizados em 
`outputs/tables` (tabelas geradas) e `outputs/results` (artefatos para consulta/apresentação).


## 1. Configuração Geral
Defina: modelos a incluir, tipo de ensemble e métrica de peso (quando weighted).
Os dados de entrada dos folds por arquitetura estão em `datas/summary_results_<modelo>/`.

In [ ]:
import os
import modules.flags as flags
from modules.runner_sa import run_sa_pipeline
from main import discover_model_csvs, resolve_paths_outputs, run_between_models
from modules.utils import log
print('[INFO] Imports concluídos.')


[INFO] Imports concluídos.


In [2]:
# Configure as flags de execução
flags.GENERATE_LEVELS = ['patient']           # opções: ['tile','image','patient']
flags.GENERATE_PLOTS_FOR = ['soft_voting']    # opções: ['hard_voting','soft_voting','weighted']
flags.ROC_DETAIL = 'macro_micro'              # opções: 'per_class' ou 'macro_micro'
flags.MAX_WORKERS = 4                         # paralelização por modelo
flags.USE_THREADS = True                      # recomendado no Windows

print('[INFO] Flags definidas:', flags.GENERATE_LEVELS, flags.GENERATE_PLOTS_FOR, flags.ROC_DETAIL)


[INFO] Flags definidas: ['patient'] ['soft_voting'] macro_micro


## 2. Descoberta dos CSVs de Folds (datas/)
Mostra quantos CSVs foram encontrados por arquitetura e um exemplo de arquivo.

In [7]:
# Descobrir CSVs por modelo e montar configuração do SA
MODELS = ['EFFNet', 'GGNet', 'MOBNet']  # ajuste conforme necessário
tables_dir, _ = resolve_paths_outputs()
models_cfgs = []

for m in MODELS:
    csvs = discover_model_csvs(m)
    print(f'{m}: {len(csvs)} CSVs encontrados')
    if csvs:
        print('Exemplo:', os.path.basename(csvs[0]))
    else:
        print('Aviso: nenhum CSV encontrado para', m)
    print('-'*60)
    
    models_cfgs.append({
        'model_name': m,
        'ensemble_type': 'soft_voting',    # ou 'hard_voting'/'weighted'
        'csv_paths': csvs,
        'save_output_base': os.path.join(tables_dir, m),
    })

print(f'[INFO] Modelos configurados: {[cfg['model_name'] for cfg in models_cfgs]}')


EFFNet: 10 CSVs encontrados
Exemplo: 0903-102424_EFF-NET_fold0_results.csv
------------------------------------------------------------
GGNet: 10 CSVs encontrados
Exemplo: 0710-161214_GoogleNet_fold0_results.csv
------------------------------------------------------------
MOBNet: 10 CSVs encontrados
Exemplo: 0707-213643_MobileNetV2_fold0_results.csv
------------------------------------------------------------
[INFO] Modelos configurados: ['EFFNet', 'GGNet', 'MOBNet']


## 3. Ensemble por Modelo (SA): Tile → Image → Patient
Nesta etapa consolidamos os folds de um mesmo modelo em três níveis.
Os resultados são salvos em `outputs/tables/<Modelo>/Ensemble_<level>_<tipo>/`.
Depois, exportamos artefatos para `outputs/results/SA/<Modelo>/<tipo>/<Level>/`.

In [ ]:
# Execução do pipeline SA
# -----------------------------------------
log("[SA] Iniciando execução com paralelização e cache interno (tile → image → patient)...")
results = run_sa_pipeline(models_cfgs)

# -----------------------------------------
# Sumário dos artefatos gerados
# -----------------------------------------
log("[SA] Sumário dos resultados gerados:")

for cfg, res in zip(models_cfgs, results):
    model_name = cfg.get('model_name', 'modelo_desconhecido')
    log(f"── Modelo: {model_name}")

    if isinstance(res, dict):
        for lvl in ['tile', 'image', 'patient']:
            if lvl in res:
                csv_path = res[lvl].get('csv', '—')
                out_dir = res[lvl].get('out_dir', '—')
                log(f"    • Nível: {lvl.capitalize()} | CSV: {csv_path} | out_dir: {out_dir}")
            else:
                log(f"    • Nível: {lvl.capitalize()} | [não gerado]")
    else:
        log(f"    ⚠ Erro: {res}")

log("[✔] SA finalizado com sucesso.")


[16:05:51] [SA] Iniciando execução com paralelização e cache interno (tile → image → patient)...


## 4. Ensemble Entre Modelos (MA)
Combina as saídas SA dos modelos selecionados.
Os resultados são salvos em `outputs/tables/Ensemble_Between_Models/...` 
e exportados para `outputs/results/MA/<tipo>/<Level>/`.

In [ ]:
# Executar ensemble entre modelos (MA) reaproveitando SA pronto
# Ajuste ensemble_type e weight_metric conforme necessário
ensemble_type = 'soft_voting'
weight_metric = 'f1_macro'

print(f'\n[MA] Rodando ensemble entre modelos: {ensemble_type} ...')
run_between_models(MODELS, ensemble_type, weight_metric)
print('[✔] MA finalizado.')



[MA] Rodando ensemble entre modelos: soft_voting ...

[Image Level] Running ensemble between models: soft_voting ...
Saved: c:\Users\rodri\OneDrive - UFPE\Área de Trabalho\meu cod\outputs\tables\Ensemble_Between_Models\ImageLevel_Ensemble_Models_soft_voting\ensemble_between_models_per_image_soft_voting.csv
Metrics: c:\Users\rodri\OneDrive - UFPE\Área de Trabalho\meu cod\outputs\tables\Ensemble_Between_Models\ImageLevel_Ensemble_Models_soft_voting\metrics_image_level_soft_voting.json

[Tile Level] Running ensemble between models: soft_voting ...
Saved: c:\Users\rodri\OneDrive - UFPE\Área de Trabalho\meu cod\outputs\tables\Ensemble_Between_Models\TileLevel_Ensemble_Models_soft_voting\ensemble_between_models_tile_level_soft_voting.csv
Metrics: c:\Users\rodri\OneDrive - UFPE\Área de Trabalho\meu cod\outputs\tables\Ensemble_Between_Models\TileLevel_Ensemble_Models_soft_voting\metrics_tile_level_soft_voting.json

[Patient Level] Running ensemble between models: soft_voting ...
Saved: c:\Use